In [1]:
!pip install swcc tqdm

     |████████████████████████████████| 54 kB 2.3 MB/s eta 0:00:011
     |████████████████████████████████| 82 kB 1.5 MB/s  eta 0:00:011
     |████████████████████████████████| 242 kB 73.7 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 150.7 MB/s eta 0:00:01
     |████████████████████████████████| 13.7 MB 76.9 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 9.2 MB/s s eta 0:00:01


In [2]:
from pathlib import Path
import re
from shutil import rmtree

import getpass
from tqdm.notebook import tqdm

from swcc.api import swcc_session
from swcc.models import (
    Dataset, GroomedSegmentation, OptimizedParticles,
    OptimizedShapeModel, Project, Segmentation, Subject
)

In [3]:
with swcc_session() as session:
    token = session.login(getpass.getpass('username'), getpass.getpass('password'))
    
    print([(d.id, d.name) for d in Dataset.list()])

username········
password········
[(78, 'ellipsoid8_16'), (79, 'ellipsoid8_3'), (88, 'ellipsoid_test'), (56, 'left_atrium'), (61, 'left_atrium_test_4')]


In [5]:
# For development, it is possible to generate a session outside of a context manager.
ctx = swcc_session(token=token)
session = ctx.__enter__()

print([(d.id, d.name) for d in Dataset.list()]) 

[(78, 'ellipsoid8_16'), (79, 'ellipsoid8_3'), (88, 'ellipsoid_test'), (56, 'left_atrium'), (61, 'left_atrium_test_4')]


In [6]:
# To connect to a different server, sepecify it as part of the session:
if False:  # Enable to run this cell
    base_url = 'http://burgess:8000/api/v1'
    with swcc_session(base_url=base_url) as session:
        token = session.login('admin@noemail.com', 'django-password')
    ctx = swcc_session(base_url=base_url, token=token)
    session = ctx.__enter__()


In [7]:
# All domain models are pydantic models that can be created in offline mode:
dataset = Dataset(name='test dataset2', license='license', description='just a test', acknowledgement='NIH')

# Note that it is missing an `id`.
dataset

Dataset(id=None, name='test dataset2', license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

In [8]:
# To generate the entity on the server, you must call its `create` method
dataset.create()

# Now its id exists
dataset

Dataset(id=89, name='test dataset2', license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

In [9]:
# Given an id, you can fetch data from the server
dataset_id = dataset.id
Dataset.from_id(dataset_id)

Dataset(id=89, name='test dataset2', license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

In [10]:
# You can also list all entities on the server using the `list` method
for d in Dataset.list():
    print(d)

id=78 name='ellipsoid8_16' license='license information' description='description' acknowledgement='acknowledgement' keywords='' contributors='' publications=''
id=79 name='ellipsoid8_3' license='license information' description='description' acknowledgement='acknowledgement' keywords='' contributors='' publications=''
id=88 name='ellipsoid_test' license='Users should acknowledge that the collection and processing of these images was supported by the National Institutes of Health' description='Unaligned ellipsoids that vary along the x-axis.                      These ellipsoids have randomized size, center, rotation, and x radii.                     Images and segmentation have spacing [1,1,2]' acknowledgement='NIH' keywords='' contributors='' publications=''
id=56 name='left_atrium' license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' sec

In [12]:
# Entities are deleted from the server using the `delete` method
# dataset.delete()

# Now we find it no longer exists
Dataset.from_id(89)

Dataset(id=89, name='test dataset2', license='license', description='just a test', acknowledgement='NIH', keywords='', contributors='', publications='')

A dataset consists of an Excel .xlsx file with a sheet named `data`.  That sheet has to contain two columns: `shape_file` and `image_file`, both of which contain corresponding lists of relative paths to .nrrd files.

Any number of projects can be aded to a dataset.  Each project consists of an Excel .xlsx file with two sheets: `data` and `optimize`.  The `data` sheet has columns `shape_file`, `groomed_file`, `alignment_file`, `local_particles_file`, and `world_particles_file`.  The shape file and groomed file columns contain relative paths to .nrrd files.  The local and world particles file columns contain relative paths to .particles files.  The alignment file column is a 12 parameter transform consisting of numbers separated by spaces.  The `optimize` sheet has columnes `key` and `value` which contain model parameters.

In [13]:
# upload the left_atrium dataset
id_re = re.compile(r'.*(CARMA\d+)\.[^/]*')
root_path = Path('data')
dataset_path = root_path / 'left_atrium-v0'

basename = 'left_atrium'
name = basename
suffix = 0
while True:
    old = Dataset.from_name(name)
    # Either get rid of an existing dataset with the same name, or create a new name for testing
    if not old:
        break
    if False:
      print('Deleting previous version of %s' % name)
      old.delete()
    suffix = suffix + 1
    name = '%s-%s' % (basename, suffix)
# TODO: Fill in with correct information
description = 'left_atrium'
acknowledgement = 'acknowledgement'

with (dataset_path / 'License.txt').open('r') as f:
    license = f.read()

dataset = Dataset(name=name, license=license, description=description, acknowledgement=acknowledgement).create()
dataset.load_data_spreadsheet(dataset_path / 'data.xlsx')
dataset.add_project(dataset_path / Path('project.xlsx'))
print(dataset.id, dataset.name)

90 left_atrium-1


In [14]:
# We can fetch a dataset by name
d = Dataset.from_name(name)
print(d.dict())

{'id': 90, 'name': 'left_atrium-1', 'license': '=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and Deformation Analysis. Academic Press, 2017. 257-298.\n\n@

In [15]:
# Print information about the dataset
for key in ('acknowledgement', 'contributors', 'description', 'id', 'keywords', 'license', 'name', 'publications'):
    print('%s: %s' % (key, getattr(dataset, key)))

acknowledgement: acknowledgement
contributors: 
description: left_atrium
id: 90
keywords: 
license: === NAMIC: CARMA Left Atrial MRI ===

If you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the 'Acknowledgments' section of your paper:

"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."

and add the following 'disclaimer':

"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."


When referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.

Joshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and Deformation Analysis. 

In [16]:
# Some components of the dataset are returned from functions
for key in ('dict', 'json', 'schema', 'schema_json'):
    print('%s: %s' % (key, getattr(dataset, key)()))
# Some components are generators    
for key in ('projects', 'segmentations', 'subjects', ): 
    for idx, value in enumerate(getattr(dataset, key)):
        print('%s:%d: %s' % (key, idx, value))


dict: {'id': 90, 'name': 'left_atrium-1', 'license': '=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and Deformation Analysis. Academic Press, 2017. 257-298

segmentations:1: id=1027 file=<swcc.models.FileType object at 0x7f1b90e659a0> anatomy_type='unknown' subject=Subject(id=733, name='CARMA0072.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Wh

segmentations:5: id=1031 file=<swcc.models.FileType object at 0x7f1b90e65730> anatomy_type='unknown' subject=Subject(id=737, name='CARMA0291.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Wh

segmentations:7: id=1033 file=<swcc.models.FileType object at 0x7f1b90e65880> anatomy_type='unknown' subject=Subject(id=739, name='CARMA0315.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Wh

segmentations:10: id=1036 file=<swcc.models.FileType object at 0x7f1b90e65940> anatomy_type='unknown' subject=Subject(id=742, name='CARMA0440.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross W

...

segmentations:38: id=1064 file=<swcc.models.FileType object at 0x7f1b913d6cd0> anatomy_type='unknown' subject=Subject(id=770, name='CARMA1305.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross W

segmentations:41: id=1067 file=<swcc.models.FileType object at 0x7f1b90e65850> anatomy_type='unknown' subject=Subject(id=773, name='CARMA1323.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross W

segmentations:44: id=1070 file=<swcc.models.FileType object at 0x7f1b913d6190> anatomy_type='unknown' subject=Subject(id=776, name='CARMA1364.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross W

segmentations:47: id=1073 file=<swcc.models.FileType object at 0x7f1b90e65f70> anatomy_type='unknown' subject=Subject(id=779, name='CARMA1399.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross W

subjects:0: id=732 name='CARMA0046.laendo_no_veins' dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.sci.utah.edu.\n\nJoshua Cates, Shireen Elhabian, Ross Whitaker. "Shapeworks: particle-based shape correspondence and visualization software." Statistical Shape and

In [17]:
# We can examine a segmentation
seg = next(dataset.segmentations)
print('id: %s\nsubject: %s\nfile: %s\nanatomy_type: %s' % (seg.id, seg.subject.name, seg.file.name, seg.anatomy_type))
# We could also get the information as a dictionary or json via
# seg.dict() or seg.json()

id: 1026
subject: CARMA0046.laendo_no_veins
file: CARMA0046.laendo_no_veins.nrrd
anatomy_type: unknown


In [18]:
# We can examine a subject
subj = next(dataset.subjects)
# We could have also gotten the subject of a segmentation:
# subj = next(dataset.segmentation).subject
print('id: %s\nname: %s\nsegmentations: %s' % (subj.id, subj.name, list(subj.segmentations)))


id: 732
name: CARMA0046.laendo_no_veins
segmentations: [Segmentation(id=1026, file=<swcc.models.FileType object at 0x7f1b90e14cd0>, anatomy_type='unknown', subject=Subject(id=732, name='CARMA0046.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of Health."\n\n\nWhen referencing ShapeWorks, please include a bibliographical reference to the paper below, and, if possible, include a link to shapeworks.s

In [19]:
# We can examine a project
proj = next(dataset.projects)
print('id: %s\nfile: %s\ndescription: %s\ngroomed segmentations: %s\nshape models: %s' % (
    proj.id, proj.file.name, description, list(proj.groomed_segmentations), list(proj.shape_models)))
# And a groomed segmentaion
print('------ groomed segmentation ------')
gseg = next(proj.groomed_segmentations)
print(gseg.dict())
# Or a shape models
print('------ shape model ------')
smod = next(proj.shape_models)
print(smod.dict())

id: 146
file: project.xlsx
description: left_atrium
groomed segmentations: [GroomedSegmentation(id=1075, file=<swcc.models.FileType object at 0x7f1b90ea64f0>, pre_cropping=None, pre_alignment=None, segmentation=Segmentation(id=1075, file=<swcc.models.FileType object at 0x7f1b90e5c280>, anatomy_type='unknown', subject=Subject(id=781, name='CARMA1404.laendo_no_veins', dataset=Dataset(id=90, name='left_atrium-1', license='=== NAMIC: CARMA Left Atrial MRI ===\n\nIf you use ShapeWorks in work that leads to published research, we humbly ask that you to cite ShapeWorks, add the following to the \'Acknowledgments\' section of your paper:\n\n"The National Institutes of Health supported this work under grant numbers NIBIB-U24EB029011, NIAMS-R01AR076120, NHLBI-R01HL135568, NIBIB-R01EB016701, and NIGMS-P41GM103545."\n\nand add the following \'disclaimer\':\n\n"The content is solely the responsibility of the authors and does not necessarily represent the official views of the National Institutes of

In [20]:
dataset.download('/tmp/example')

In [21]:
dataset_path = '/tmp/example'
root_path = Path(dataset_path)

# TODO: Fill in with correct information
name = description = 'left_atrium_test_2'
old = Dataset.from_name(name)
if old:
  print('Deleting previous version of %s' % name)
  old.delete()

acknowledgement = 'acknowledgement'
license = 'license'

d2 = Dataset(name=name, license=license, description=description, acknowledgement=acknowledgement).create()
d2.load_data_spreadsheet(root_path / 'project.xlsx')

Dataset(id=91, name='left_atrium_test_2', license='license', description='left_atrium_test_2', acknowledgement='acknowledgement', keywords='', contributors='', publications='')